In [ ]:
import pandas as pd
from google.genai import types
from google import genai
import re, os, json
from jiwer import wer, cer
from dotenv import load_dotenv
from rapidfuzz import process
from scipy.stats import kendalltau

load_dotenv(".env")
API_KEY = os.getenv("GEMINI_API_KEY")
# print(API_KEY)

client = genai.Client(
    api_key=API_KEY
)

In [2]:
with open('../../evaluation/eval_list.txt', 'r') as file:
    content = file.read()

eval_list = []
for img_name in content.split('\n'):
    eval_list.append(img_name.split('.')[0])

eval_list = eval_list[83:]
print(len(eval_list), eval_list[0])

17 187


In [11]:
print(eval_list)

['187', '615', '1', '295', '244', '286', '8', '120', '5', '586', '177', '47', '471', '494', '658', '535', '450']


In [3]:
start = 424
n = 425
text_list = [f'../data/raw/ocr_result/ocr_{x}.txt' for x in eval_list]
print(len(text_list), text_list[0], text_list[-1])

gt_list = [f'../data/raw/ground_truth/ocr_{x}.txt' for x in eval_list]
print(len(gt_list), gt_list[0], gt_list[-1])

17 ../data/raw/ocr_result/ocr_187.txt ../data/raw/ocr_result/ocr_450.txt
17 ../data/raw/ground_truth/ocr_187.txt ../data/raw/ground_truth/ocr_450.txt


In [4]:
def return_prompt(row):
    prompt = (
        "Teks berikut adalah hasil OCR:\n\n"
        f"{row}\n\n"
        "Instruksi:\n"
        "1. Jangan susun ulang teks.\n"
        "2. Perbaiki typo dan kesalahan yang penyebabnya adalah OCR.\n"
        "3. Jangan menambah, mengurangi, atau mengubah informasi apa pun.\n"
        "4. Jangan menambah komentar, penjelasan, atau catatan.\n"
        "5. Output harus berupa teks final saja, tanpa markdown, tanpa format tambahan.\n"
        "6. Jangan melakukan asumsi atau mengisi bagian teks yang hilang.\n"
        "7. Teks output **harus berisi kata-kata yang sama dengan input**, kecuali kata yang memang diperbaiki karena typo.\n"
        "8. Jangan mengubah struktur kalimat secara berlebihan—hanya perbaiki ejaan.\n\n"
        "PERINTAH PENTING:\n"
        "• Jangan membuat kalimat baru.\n"
        "• Jangan menghilangkan kata.\n"
        "• Jangan melakukan halusinasi.\n"
        "• Kembalikan hanya teks yang sudah diperbaiki."
    )
    return prompt

In [5]:
baseline = []
trained = []

def pmr(gt, pred):
    gt_words = gt.split()
    pred_words = pred.split()
    length = min(len(gt_words), len(pred_words))
    matches = sum(1 for i in range(length) if gt_words[i] == pred_words[i])
    return matches / length

In [6]:
import time
from tqdm import tqdm

for index in tqdm(eval_list, desc='Correcting typos'):
    time.sleep(5)
    row = ""
    with open(f"../../yolo/yolo_res/res_{index}.txt", "r", encoding="utf-8", errors="ignore") as file:
        row = file.read()

    try:
        model = "gemini-2.5-flash"
        contents = [
            types.Content(
                role="user",
                parts=[
                    types.Part.from_text(text=return_prompt(row)),
                ],
            ),
        ]
        generate_content_config = types.GenerateContentConfig(
            response_mime_type="text/plain",
        )

        response_text=""
        for chunk in client.models.generate_content_stream(
            model=model,
            contents=contents,
            config=generate_content_config,
        ): response_text += chunk.text

        # print(index)
        # print(response_text)
        
        with open(f"./final_LLM_res/res_{index}.txt", "w",encoding="utf-8", errors="ignore") as file:
            file.write(response_text)

    except Exception as e:
        print(f"Row - Error processing column :", e)

Correcting typos:  35%|███▌      | 6/17 [02:05<03:35, 19.60s/it]

Row - Error processing column : 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'Visibility check was unavailable. Please retry the request and contact support if the problem persists', 'status': 'UNAVAILABLE'}}


Correcting typos: 100%|██████████| 17/17 [06:01<00:00, 21.24s/it]


In [7]:
# evaluate(gt, row, True)
# evaluate(gt, response_text, False)

In [8]:
# with open("eval_baseline.txt", "a") as file:
#     file.write(str(baseline))
# with open("eval_trained.txt", "a") as file:
#     file.write(str(trained))

In [9]:
# print(baseline)

In [10]:
# print(trained)